In [ ]:
import gspread

def get_first_row_from_google_sheet():
    # Авторизация по сервисному аккаунту
    gc = gspread.service_account(filename="config_gkey.json")
    
    # Открываем таблицу и первый лист
    wks = gc.open("web_parsing_baza_knig").sheet1
    
    # Получаем первую строку (индексация начинается с 1!)
    first_row = wks.row_values(1)
    
    # Если нужно первое значение первой строки:
    first_value = wks.cell(1, 1).value
    
    return first_row

# Вызов функции
print(get_first_row_from_google_sheet())


In [ ]:
import gspread
import pandas as pd

def update_google_sheet(df: pd.DataFrame, mode: str = "append"):
    """
    Обновление Google Sheets из DataFrame
    
    mode:
      - "append"   → добавить в конец
      - "prepend"  → добавить в начало (после заголовков)
      - "overwrite"→ полностью перезаписать
    """
    gc = gspread.service_account(filename="config_gkey.json")
    wks = gc.open("web_parsing_baza_knig").sheet1

    rows = df.values.tolist()  # DataFrame → список списков

    if mode == "append":
        # Добавляем в конец
        wks.append_rows(rows, value_input_option="RAW")

    elif mode == "prepend":
        # Вставляем пустые строки под заголовками
        n_rows = len(df)
        wks.insert_rows([''] * n_rows, row=2)
        # Обновляем начиная с A2
        wks.update("A2", rows)

    elif mode == "overwrite":
        # Полностью перезаписываем таблицу (с заголовками)
        wks.clear()
        header = [df.columns.tolist()]
        wks.update("A1", header + rows)

    else:
        raise ValueError("mode должен быть 'append', 'prepend' или 'overwrite'")
)
# Добавить в начало (сдвигая старые данные вниз)
update_google_sheet(df, mode="prepend")